# Deleting entity sets

(Tutorial adapted from Joke Durnez)

In this notebook, we will:

1. Create an entity set
2. Add a few datapoints to the entity set (these functions are unstable at this point)
3. Select entity keys based on a condition, i.e. does the primary key contain the word "Woman"
4. Delete those entities
5. Delete all entities in the entity set
6. Delete the entity set

To follow along, you will need to install [olpy](https://github.com/openlattice/olpy) and [openlattice](https://github.com/Lattice-Works/api-clients/tree/master/python). [This](https://help.openlattice.com/article/104-overview-and-installation-of-api-clients) is a detailed setup instructions for setting up the openlattice module.

## Set up

In [1]:
import openlattice
import olpy
import pandas as pd
import numpy as np
from collections import Counter
from pprint import pprint
import uuid

In [2]:
baseurl = 'https://api.openlattice.com'

configuration = openlattice.Configuration()
configuration.host = baseurl
configuration.access_token = olpy.get_jwt(base_url = configuration.host)

edmAPI = openlattice.EdmApi(openlattice.ApiClient(configuration))
entitySetsAPI = openlattice.EntitySetsApi(openlattice.ApiClient(configuration))
permissionsAPI = openlattice.PermissionsApi(openlattice.ApiClient(configuration))
dataAPI = openlattice.DataApi(openlattice.ApiClient(configuration))
integrationAPI = openlattice.DataIntegrationsApi(openlattice.ApiClient(configuration))
orgAPI = openlattice.OrganizationsApi(openlattice.ApiClient(configuration))

## 1. Create entity set

First create the entity set and get the entity set ID. To find the organization Id, go to the [Orgs App](https://staging.openlattice.com/orgs/#/orgs) and find your desired organization. Then, go to database details within the organization. You should be able to see the organization ID there.

In [3]:
# get the entity type
entity_type_id = edmAPI.get_entity_type_id(namespace = "general", name = "person")

entset = {
        "entityTypeId": entity_type_id,
        "name": "Test12345",
        "title": "Test12345",
        "contacts": ["yue@openlattice.com"],
        "organizationId": "961d293e-d244-4939-ad1b-69b356038825" # Yue's Test org
    }

entitySetsAPI.create_entity_sets(entity_set=[entset])

ApiException: (500)
Reason: Server Error
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 20 Jan 2021 21:19:32 GMT', 'Content-Type': 'application/json;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Server': 'nginx/1.18.0 (Ubuntu)', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Frame-Options': 'DENY', 'Strict-Transport-Security': 'max-age=31536000'})
HTTP response body: {"errors":[{"error":{},"message":"Test12345: Type Test12345 already exists."}]}


In [4]:
entitysetid = entitySetsAPI.get_entity_set_id('Test12345')
entitysetid

'0f898abd-edfe-4212-8642-74db49efa474'

## 2. Add data

Next step: we'll write the data. The steps to this:

1. For each primary key (in this case, the name), create the entity key IDs if they do not exist already.
2. To add data, you will need to create a dictionary that maps the entity key IDs to the corresponding property type ID and data, i.e. `{entity_key_id} : {property_type_id: [data]}}`
3. Use data API to update entities in the dataset

In [27]:
data = pd.DataFrame({'name': ['Woman1', 'Woman2', 'Child1', 'Child2']})
data_entity_keys = [openlattice.EntityKey(entitysetid, x) for x in data['name']]
entity_key_ids = integrationAPI.get_entity_key_ids(data_entity_keys)
entity_key_ids

['b89d0000-0000-0000-8000-00000001f385',
 'b8ab0000-0000-0000-8000-00000001e285',
 'b8ac0000-0000-0000-8000-00000001f2b0',
 'b8ac0000-0000-0000-8000-00000001f2b1']

In [31]:
prep_data = {}

for idx, row in data.iterrows():
    prep_data[entity_key_ids[idx]] = {
        edmAPI.get_property_type_id('nc', 'SubjectIdentification'):
        [row['name']]
    }
    
prep_data

{'b89d0000-0000-0000-8000-00000001f385': {'5260cfbd-bfa4-40c1-ade5-cd83cc9f99b2': ['Woman1']},
 'b8ab0000-0000-0000-8000-00000001e285': {'5260cfbd-bfa4-40c1-ade5-cd83cc9f99b2': ['Woman2']},
 'b8ac0000-0000-0000-8000-00000001f2b0': {'5260cfbd-bfa4-40c1-ade5-cd83cc9f99b2': ['Child1']},
 'b8ac0000-0000-0000-8000-00000001f2b1': {'5260cfbd-bfa4-40c1-ade5-cd83cc9f99b2': ['Child2']}}

In [32]:
dataAPI.update_entities_in_entity_set(
    entity_set_id = entitysetid,
    type = "Replace",
    request_body = prep_data
)

4

#### Now load the data... And there it is!

This loads a dataset that's a list of dictionaries. The field openlattice.@id is a unique entity key for each entity and is generated uniquely by OpenLattice during integration. The field nc.SubjectIdentification is a standardized field for the entity type general.person. To see what property types exist in an entity type, check out our [EDM page](https://openlattice.com/edm/#/propertyTypes). For a more thorough explanation of entity models, check out [this help page](https://help.openlattice.com/article/20-the-openlattice-standardized-entity-data-model).

In [33]:
data = dataAPI.load_entity_set_data(entitysetid)
pprint(data)

[{'nc.SubjectIdentification': ['Child1'],
  'openlattice.@id': ['b8ac0000-0000-0000-8000-00000001f2b0']},
 {'nc.SubjectIdentification': ['Woman1'],
  'openlattice.@id': ['b89d0000-0000-0000-8000-00000001f385']},
 {'nc.SubjectIdentification': ['Woman2'],
  'openlattice.@id': ['b8ab0000-0000-0000-8000-00000001e285']},
 {'nc.SubjectIdentification': ['Child2'],
  'openlattice.@id': ['b8ac0000-0000-0000-8000-00000001f2b1']}]


### 3. Now: find the entity key IDs that satisfies the condition

In [34]:
entitykeys_to_delete = []
for datapoint in data:
    for value in datapoint['nc.SubjectIdentification']:
        # condition: if we see the word Woman in the value....
        if "Woman" in value:
            entitykeys_to_delete.append(datapoint['openlattice.@id'][0])
            
entitykeys_to_delete

['b89d0000-0000-0000-8000-00000001f385',
 'b8ab0000-0000-0000-8000-00000001e285']

## 4. And (selectively) delete!

In [35]:
dataAPI.delete_entities(
    entity_set_id = entitysetid, 
    type = "Soft", 
    request_body = entitykeys_to_delete)

2

Now we see that only `Child1` and `Child2` still exist!

In [36]:
pprint(data)

[{'nc.SubjectIdentification': ['Child1'],
  'openlattice.@id': ['b8ac0000-0000-0000-8000-00000001f2b0']},
 {'nc.SubjectIdentification': ['Child2'],
  'openlattice.@id': ['b8ac0000-0000-0000-8000-00000001f2b1']}]


## 5. Delete all of the data in the entity set

To indiscriminately delete data from the dataset you can call the `delete_all_entities_from_entity_set`. If for whatever reason, the entity set you created had an issue, this is useful. And now, you can see it returns an empty list.

In [38]:
dataAPI.delete_all_entities_from_entity_set(entity_set_id = entitysetid, type = "Hard")

In [39]:
data = dataAPI.load_entity_set_data(entitysetid)
pprint(data)

[]


## 6. Delete the entity set

If the entity set is no longer needed, you can get rid of all of the data and metadata associated with the entity set. If we try to load the dataset, it'll return an error.

In [40]:
entitySetsAPI.delete_entity_set(entitysetid)

In [42]:
# data = dataAPI.load_entity_set_data(entitysetid)

## Bonus: deleting multiple entity sets at a time

Sometimes, you want to delete a bunch of entity sets. The easiest way to do so is to just run a for loop.

In [ ]:
ent_sets_to_delete = ['e1', 'e2']

for ent in ent_sets_to_delete:
    ent_id = entitySetsAPI.get_entity_set_id(ent)
    entitySetsAPI.delete_entity_set(ent_id)
    print(f"{ent} deleted!")